In [632]:
import TableParser
import pandas as pd
import tempfile, zipfile
import os
import numpy as np
import datetime
from collections import Counter

In [633]:
class TestTableParser(TableParser.TableParserItem):
    def __init__(self):
        pass
    
    def parse(self, series: pd.Series):
        mistake = {
            "00:00:00":'','0:00':'',
            'Jan':'01','Feb': '02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sept':'09','Oct':'10','Nov':'11','Dec':'12',
            'o': '0','l': '1','b': '6','g': '9','q': '9','t': '7','v': '5','G': '6','F': '7','Z': '2','Q': '2','B': '8','O': '0','D': '0','E': '3','A': '4','S': '5',
            r'[ -/]|[:-@]|[\[-_]|[{-~]': '-',
        }
        for key, val in mistake.items():
            series = series.str.strip().str.replace(key, val, regex=True)
        return (pd.to_datetime(series, errors="coerce"), [])

    def fix_xlsx(self, in_file):
        tmpfd, tmp = tempfile.mkstemp(dir=os.path.dirname(in_file))
        os.close(tmpfd)
        filename = '[Content_Types].xml'
        data = ''
        with zipfile.ZipFile(in_file, 'r') as zin:
            with zipfile.ZipFile(tmp, 'w') as zout:
                for item in zin.infolist():
                    if item.filename != filename:
                        zout.writestr(item, zin.read(item.filename))
                    else:
                        data = zin.read(filename).decode()
        os.remove(in_file)
        os.rename(tmp, in_file)
        data = data.replace('/xl/sharedStrings.xml', '/xl/SharedStrings.xml')
        with zipfile.ZipFile(in_file, mode='a', compression=zipfile.ZIP_DEFLATED) as zf:
            zf.writestr(filename, data)
            
    def orientation_determine(self,df):
        #True - Y axis table
        #False - X axis table
                                    
#         if (df.shape[0] > df.shape[1] and pd.Series(df.index.tolist()).dtypes == int):
#             return True
#         else: return False
        test_list =[[0,0]]
        count_X_axis,count_Y_axis,main_axis = 0,0,True
        for i in range(df.shape[0]):
            if len(test_list) == 11: 
                break
            for j in range(df.shape[1]):
                if  type(str(df.iloc[i,j])) == type("string") and len(str(df.iloc[i,j])) >=10:
                    if type(testTableParser.parse(pd.Series([str(df.iloc[i,j])]))[0][0]) == type(pd.to_datetime('31.01.2002')):
                        test_list.append([i,j])
                        if test_list[-1][1] > test_list[-2][1]:
                            count_X_axis += 1
                        if test_list[-1][0] > test_list[-2][0]:
                            count_Y_axis += 1  
                        if len(test_list) == 11:
                            break
        if (count_Y_axis  > count_X_axis):
            return True
        else: return False
        

    def set_type_of_cell(self,df):
        def func(x):
                try:
                    return int(x)
                except Exception:
                    try:
                        return float(x)
                    except Exception:
                        return x
        
        for i in range(df.shape[1]):
            before_change = df.iloc[:,i].astype(str)
            df.iloc[:,i] = before_change.apply(func)
#             print(df.iloc[:,i])
        return df
            
    
    def all_action(self,df): # <-----    drive method
        orientation = self.orientation_determine(df)
        print(orientation)

        if orientation== False:
            df = df.T.copy()
#         df = self.set_type_of_cell(df).copy()

#         print([type(each) for each in df.iloc[:,4]])
# <class 'datetime.datetime'>
        return self.set_type_of_cell(df)



In [634]:
# df = pd.read_excel(open('file44.xlsx', 'rb'),header=None,sheet_name='Стадо 1 ')
# df = pd.read_csv("file44.csv",encoding='unicode_escape',header=None)
# testTableParser.fix_xlsx('file1.xlsx')
testTableParser = TestTableParser()
df = pd.read_excel("file1.xlsx",header=None)
print(df.shape)
df = testTableParser.all_action(df)
df
# df.dtypes.value_counts()


(31, 927)
False


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,NaN,"ПАО ""Птицефабрика ""Боровская""",Цех №2,NaN,Показатели,Дата на начало периода,Наличие кур-несушек\nна начало периода,Пало,NaN,NaN,...,Среднемесячное поголовье,Яиц на начальную\nкурицу-несушку,NaN,Яиц на среднюю\nкурицу-несушку,NaN,Яйценоскость,NaN,Средний вес 1-ой головы,Однородность %,Средний вес 1-го яйца
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,норм,факт,норм,...,NaN,норм,факт,норм,факт,норм,факт,NaN,NaN,NaN
2,NaN,NaN,Птичник: Пт № 16,NaN,ЕдИзм,NaN,гол.,гол.,гол.,%,...,гол.,шт.,шт.,шт.,шт.,%,%,кг,NaN,г
3,NaN,Показатели продуктивного периода 1 Марта 201...,NaN,NaN,NaN,01.03.2016,24490,98,10,0.4,...,875,20.2,0.68,21.0,18.93,75.0,67.62,2.018,75.0,65.6
4,NaN,NaN,NaN,NaN,NaN,02.03.2016,24486,98,4,0.4,...,875,20.2,0.69,21.0,19.35,75.0,69.1,2.018,75.0,65.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922,NaN,NaN,NaN,NaN,NaN,10.12.2018,29999,150,10,0.5,...,1071,18.1,0.73,19.0,16.8,67.86,60.0,1.93,80.0,NaN
923,NaN,NaN,NaN,NaN,NaN,11.12.2018,29995,150,4,0.5,...,1071,18.1,0.71,19.0,16.13,67.86,57.61,1.93,80.0,NaN
924,NaN,NaN,NaN,NaN,NaN,12.12.2018,18092,90,5,0.5,...,646,18.1,0.65,19.0,24.51,67.86,87.55,1.93,80.0,NaN
925,NaN,NaN,NaN,NaN,NaN,13.12.2018,7119,107,7,1.5,...,254,17.0,0.29,18.0,27.61,64.29,98.61,1.93,80.0,NaN


In [666]:
#######################
# start of useful data#
#######################
df = pd.read_excel("file22.xlsx",header=None)
start_point = 0
for i in range(df.shape[0]):
    if len(pd.to_numeric(df.iloc[i,:],errors='coerce').dropna()) != 0:
        print("finded in :"+ str(i))
        start_point = i
        break
df_data = df.iloc[start_point:,:]
df_data

finded in :4


,0,1,2,3,4,5,6,7,8,9,...,173,174,175,176,177,178,179,180,181,182
4,0,0,2019-12-28 00:00:00,8471,NaN,NaN,NaN,41.1,NaN,338.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,1,2019-12-29 00:00:00,8467,NaN,NaN,NaN,NaN,NaN,338.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,2,2019-12-30 00:00:00,8461,NaN,NaN,NaN,NaN,NaN,338.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,3,2019-12-31 00:00:00,8451,NaN,NaN,NaN,NaN,NaN,338.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,4,2020-01-01 00:00:00,8446,NaN,NaN,NaN,NaN,NaN,337.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,499,496,2021-05-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,500,497,2021-05-11 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529,501,498,2021-05-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,502,499,2021-05-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [545]:
df = pd.read_excel("file44.xlsx",header=None)
s = pd.Series([234.0,32.9,1,4,14])
# s = pd.to_numeric(df.iloc[:,0],errors='coerce').dropna()
s = s.convert_dtypes()
s.map(type)
# s.dtype == pd.Float64Dtype()
# s.dtype == pd.Int64Dtype()

0    <class 'float'>
1    <class 'float'>
2    <class 'float'>
3    <class 'float'>
4    <class 'float'>
dtype: object

In [770]:
######################
# define column type #
######################
columns_types = {}
for j in range(df_data.shape[1]):
    baff = pd.to_numeric(df_data.iloc[:,j],errors='coerce').dropna()
    if baff.convert_dtypes().sum() != 0:
        if baff.convert_dtypes().dtype == pd.Float64Dtype():
            columns_types[j] = 'float'
        else: columns_types[j] = 'int'
    else:
        if type(testTableParser.parse(df_data.iloc[:,j].astype(str))[0][start_point]) == type(pd.to_datetime('31.01.2002')):
            columns_types[j] = 'date'
        elif baff.convert_dtypes().dtype == pd.BooleanDtype():
            columns_types[j] = 'bool'
        else: columns_types[j] = 'str'
# columns_types

In [799]:
######################
# define table in df #
######################
null_list = []
index_list= []
for j in range(df_data.shape[1]):
    if columns_types[j] == 'int':
        buff = pd.to_numeric(df_data.iloc[:,j],errors='coerce')
        null_list.append(buff)
int_df = pd.concat(null_list,axis=1)
count = Counter(columns_types.values())
for i in range(int_df.shape[0]):
    if int_df.iloc[i,:].isnull().sum() == count.get('int'):
        index_list.append(i)
print(index_list)
print(int_df.shape[1])
df_data.iloc[index_list,:]

[149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]
110


,0,1,2,3,4,5,6,7,8,9,...,173,174,175,176,177,178,179,180,181,182
153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,NaN,Стадо РМ №1,NaN,NaN,1 партия,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,Воз-ст 1 партии,Воз-ст 2 партии,Показатель,1 корпус,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [670]:
s.apply(type).value_counts()
print([type(each) for each in df.iloc[:,4]])

<class 'str'>      6
<class 'float'>    1
dtype: int64

In [781]:
all_null


,0,1,3,4,5,6,8,10,14,15,...,163,164,166,171,172,173,174,175,176,178
4,0.0,0.0,8471.0,NaN,NaN,NaN,NaN,NaN,45.0,8470.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,1.0,8467.0,NaN,NaN,NaN,NaN,NaN,45.0,8467.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2.0,2.0,8461.0,NaN,NaN,NaN,NaN,NaN,46.0,8465.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3.0,3.0,8451.0,NaN,NaN,NaN,NaN,NaN,46.0,8463.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4.0,4.0,8446.0,NaN,NaN,NaN,NaN,NaN,46.0,8462.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,499.0,496.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4523.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528,500.0,497.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529,501.0,498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530,502.0,499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
data = data.astype('int64',errors="ignore")

In [545]:
s = pd.Series(['31.01.2002'])
type(testTableParser.parse(pd.Series(['31.01.2002']))[0][0])

pandas._libs.tslibs.timestamps.Timestamp

In [546]:
type(pd.to_datetime('31.01.2002'))

pandas._libs.tslibs.timestamps.Timestamp

In [577]:
df.iloc[:,5].apply(type).value_counts()


<class 'str'>    2824
Name: 5, dtype: int64

In [688]:
# s = pd.Series([np.nan, np.nan, np.nan])
s = pd.Series(["aa","bb",'cc'])
dfn = s.convert_dtypes()
dfn

0    aa
1    bb
2    cc
dtype: string

AttributeError: module 'pandas' has no attribute 'string'

In [554]:
s = pd.Series(['A', 'B', 'Aaba', 'Baca', np.nan, 'CABA', 'cat'])
